In [6]:
from collections import defaultdict
from itertools import combinations
from pathlib import Path
from gensim.models import Word2Vec
import networkx as nx
import pandas as pd
import numpy as np
from matplotlib import pyplot

In [7]:
# Reads csv file into a pandas dataframe
articles = pd.read_csv(Path("data") / "articles.csv")

# Adds a new columns called node_id which corresponds to the index
articles["node_id"] = articles.index

# Make List into an array
articles["lists"] = articles["lists"].str.split("; ")

### REMOVE THIS BEFORE SUBMISSION HERE I AM TAKING A SAMPLE
#articles = articles.sample(n=10, random_state=42)
test_data = pd.read_csv(Path("data") / "test_data.csv")
train_data = pd.read_csv(Path("data") / "train_data.csv")

In [8]:
medium_graph = nx.Graph()
medium_graph.add_nodes_from(articles["node_id"].to_list())

list_to_nodes = defaultdict(set)
for _, row in articles[["node_id", "lists"]].iterrows():
    for l in row["lists"]:
        list_to_nodes[l].add(row["node_id"])

for node_ids in list_to_nodes.values():
    medium_graph.add_edges_from(combinations(node_ids, 2))

Let us make the walks now to get the node embeddings. 

In [13]:
# Performing Random Walk
def random_walks(graph: nx.Graph, num_walks: int, walk_length: int) -> np.ndarray:
    result = []

    for node in graph.nodes():
        for i in range(num_walks):
            walk = [node]
            for j in range(walk_length - 1):
                current_node = walk[-1]
                neighbors_list = list(graph.neighbors(current_node))
                
                if len(neighbors_list) == 0:
                    walk.append(node)
                    continue        

                # Randomly choose a neighbor
                index = np.random.randint(len(neighbors_list))
                next_node = neighbors_list[index]
                walk.append(next_node)
                
             # Pad shorter walks with None as we want to create equal length np arrays   
            if len(walk) < walk_length:
                walk.extend([node] * (walk_length - len(walk)))
            result.append(walk)

    result = [[str(w) for w in walk] for walk in result]
    return result


In [15]:
print(medium_graph)

depth = 100
walks = random_walks(medium_graph, num_walks= 5, walk_length = depth)



Graph with 27718 nodes and 2014162 edges


Now let us use the word2vec model to embed these walks. 

In [17]:
# We can take the context vector to be the mode at each index in the walk.

model = Word2Vec(
    walks,
    vector_size=128,  # Dimensionality of the embeddings
    window=5,         # Context window size
    min_count=0,      # Ignore words with frequency below this
    sg=1,             # Use skip-gram model
    workers=4,        # Number of threads to use
    epochs=10         # Number of iterations over the corpus
)

model.save("word2vec_model.model")

Let us load the model

In [9]:
model = Word2Vec.load("word2vec_model.model")



In [10]:
graphS = nx.relabel_nodes(medium_graph, lambda x: str(x))

In [11]:
nodeEmbeddings = {}

for node in medium_graph.nodes():

    v = model.wv[str(node)]
    nodeEmbeddings[node] = v
    # print(v.shape)

# 128 dim vector embedding for each node

# kNN Classifier training in the embedding space. 

Now that we have the embedding, we can trian a kNN classifier given the training data. 
We will do nearest neighbours in the embedding space and use euclidean distance.


In [12]:
# print(nodeEmbeddings.keys())

# We can train on the train data

trainEmbeddings = []
trainLabels = train_data["label"].to_list()

for node_id in train_data["node_id"]:
    trainEmbeddings.append(nodeEmbeddings[node_id])

trainEmbeddings = np.array(trainEmbeddings)
trainLabels = np.array(trainLabels)


In [35]:
k = 5

def distance(p1,p2):
    return np.linalg.norm(p1-p2)


def getMajority(labels):
    # Count occurrences of each label
    label_counts = {}
    for label in labels:
        label_counts[label] = label_counts.get(label, 0) + 1
    
    # Sort labels by count in descending order
    sorted_labels = sorted(label_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Return the n most common labels
    return [label for label, _ in sorted_labels[:k]]


def kNN(nod_id):
    # We find the nearest k neighbours in embedding space
    majorityLabel = None

    v = np.array(nodeEmbeddings[nod_id])

    distances = np.array([distance(v,e1) for e1 in trainEmbeddings])

    firstK = np.argsort(distances)[:k]

    closestKLabels = trainLabels[firstK]

    majorityLabel = getMajority(closestKLabels)[0]

    return majorityLabel

In [39]:
# We can now use the kNN on the test data

testEmbeddings = []
testActualLabels = test_data["label"].to_list()

for node_id in test_data["node_id"]:
    testEmbeddings.append(nodeEmbeddings[node_id])

testEmbeddings = np.array(testEmbeddings)



totalCount = 0
correct = 0

i = 0
for node_id in test_data["node_id"]:
    prediction = kNN(test_data["node_id"][i])
    actual = testActualLabels[i]
    totalCount += 1
    if prediction == actual:
        correct += 1
    i += 1



accuracy = correct/totalCount



In [40]:
print(accuracy)

0.8007594936708861
